# Sampled power analysis

In [1]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import itertools
import warnings

import os
import pickle as pkl
%matplotlib inline
import imp
import statsmodels.api as sm

/tmp/ipykernel_2707/2212651623.py:13: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
import sys
sys.path.append('/home/ubuntu/Github/scrna-parameter-estimation/')
sys.path.append('/home/ubuntu/Github/misc-seq/miscseq/')
# import encode
import memento
# import run_memento

In [3]:
data_path  = '/Data/lupus/'

In [4]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", ['turquoise', 'silver', 'slategrey'])


import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'medium',
         'axes.titlesize':'medium',
         'figure.titlesize':'medium',
         'xtick.labelsize':'small',
         'ytick.labelsize':'small'}
pylab.rcParams.update(params)

### Read OneK1K hits

In [5]:
onek_replication = pd.read_csv(data_path + 'filtered_onek_eqtls.csv').query('cg_cov == "T4"')

In [6]:
onek1k_hit_genes = onek_replication.gene.drop_duplicates().tolist()

### Setup pseudobulk and metadata

In [10]:
ct = 'T4'

In [7]:
pos_asian = pd.read_csv(data_path + 'mateqtl_input/asian_genos.tsv', sep='\t', index_col=0).T

In [8]:
cov = pd.read_csv(data_path + 'mateqtl_input/asian_mateqtl_cov.txt', sep='\t', index_col=0).T

In [14]:
pb = pd.read_csv(data_path + 'pseudobulk/asian_{}_sum.csv'.format(ct), sep='\t', index_col=0)
norm_pb = pd.read_csv(data_path + 'pseudobulk/asian_{}.csv'.format(ct), sep='\t', index_col=0)

### Run t-test 

In [87]:
pv_list

array(2980)

In [95]:
pv_list = np.zeros(onek_replication.shape[0])
gene_idx = 0
for idx, row in onek_replication.iterrows():
    
    gene = row['gene']
    snp = row['rsid']
    
    _, pv_list[gene_idx] = stats.ttest_ind(
        norm_pb[gene].values[pos_asian[snp] > 0],
        norm_pb[gene].values[pos_asian[snp] ==0])
    gene_idx += 1
ttest_result = onek_replication[['gene', 'rsid']].copy()
ttest_result['ttest_pval'] = pv_list

ttest_result.to_csv(data_path + 'ttest_output/asian_T4.txt', sep='\t', index=False)

### Run GLM

In [ ]:
pv_list = np.zeros(onek_replication.shape[0])
gene_idx = 0
for idx, row in onek_replication.iterrows():
    
    gene = row['gene']
    snp = row['rsid']
    
    design = sm.add_constant(pd.concat(
    [
        pos_asian[[snp]], 
        cov
    ],
    axis=1))
    
    pos_asian[snp]
    try:
        res = sm.NegativeBinomial(pb[gene],design).fit()
        pv_list[gene_idx] = res.pvalues[snp]
    except:
        pv_list[gene_idx] = 1
    gene_idx += 1
    
glm_result = onek_replication[['gene', 'rsid']].copy()
glm_result['glm_pval'] = pv_list

glm_result.to_csv(data_path + 'glm_output/asian_T4.txt', sep='\t', index=False)

In [143]:
(glm_result.query('glm_pval < 1')['glm_pval'] < 0.05).mean()

0.3334597875569044